<a href="https://colab.research.google.com/github/natalievolk/LearnAI/blob/main/LearnAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# pillow to load image
from PIL import Image
import cv2
import requests
from io import BytesIO
#import urllib2



In [90]:
# training data
url_train = 'https://raw.githubusercontent.com/natalievolk/LearnAI/main/book30-listing-train.csv'
data_train = pd.read_csv(url_train, index_col=False, usecols=[2,5], encoding = "ISO-8859-1")
data_train.columns = ['jpg_url', 'classification']


# testing data
url_test = 'https://raw.githubusercontent.com/natalievolk/LearnAI/main/book30-listing-test.csv'
data_test = pd.read_csv(url_test, index_col=False, usecols=[2,5], encoding = "ISO-8859-1")
data_test.columns = ['jpg_url', 'classification']

data_train.head()
data_test.head(10)

,jpg_url,classification
0,http://ecx.images-amazon.com/images/I/510l0qhi...,28
1,http://ecx.images-amazon.com/images/I/51p5aUY%...,1
2,http://ecx.images-amazon.com/images/I/51pum0eW...,21
3,http://ecx.images-amazon.com/images/I/51PuTPPC...,0
4,http://ecx.images-amazon.com/images/I/51uNnIlI...,15
5,http://ecx.images-amazon.com/images/I/51VIlqRy...,10
6,http://ecx.images-amazon.com/images/I/41n7iZq-...,23
7,http://ecx.images-amazon.com/images/I/41xLYt-v...,9
8,http://ecx.images-amazon.com/images/I/51KyLXcz...,4
9,http://ecx.images-amazon.com/images/I/51Jku59B...,25


In [136]:
mini_train_size = 10
mini_train = data_train.head(mini_train_size)

mini_test_size = 5
mini_test = data_test.head(mini_test_size)

pixel_size = 400

In [137]:
# MAKING TRAINING SET

dataset = []
 
for index, image_name in enumerate(mini_train['jpg_url']):
    response = requests.get(image_name)
    img = Image.open(BytesIO(response.content)).resize((pixel_size, pixel_size))
    #print(img.size) # output should be (400, 400) if the resizing worked
    # should we crop the image??
    pix = np.array(img)
    '''
    # this gets rid of all corrupted images (ask varun!!)
    if len(pix.shape) != 3:
      mini_train_size -= 1
      mini_test_size -= 1
      mini_test.drop(index)
      continue
    '''
    print(pix.shape)
    dataset.append((pix))

train_x = np.array(dataset)

train_y = mini_train['classification'].values
train_y
train_x.shape

(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)


(10, 400, 400, 3)

In [138]:
# CONVERTING AND RESHAPING TRAINING DATA
# converting training images into torch format
train_x = train_x.reshape(mini_train_size, 3, pixel_size, pixel_size)
train_x  = torch.from_numpy(train_x)

# converting the target into torch format
train_y = train_y.astype(int);
train_y = torch.from_numpy(train_y)

# shape of training data
train_x.shape, train_y.shape

(torch.Size([10, 3, 400, 400]), torch.Size([10]))

In [139]:
# MAKING TESTING SET
dataset = []
 
for image_name in mini_test['jpg_url']:
    response = requests.get(image_name)
    img = Image.open(BytesIO(response.content)).resize((pixel_size, pixel_size))
    print(img.size) # output should be (400, 400) if the resizing worked
    # should we crop the image??
    pix = np.array(img)
    dataset.append((pix))

test_x = np.array(dataset)

test_y = mini_test['classification'].values
test_y
test_x.shape

(400, 400)
(400, 400)
(400, 400)
(400, 400)
(400, 400)


(5, 400, 400, 3)

In [140]:
# CONVERTING AND RESHAPING TESTING DATA
# converting training images into torch format
test_x = test_x.reshape(mini_test_size, 3, pixel_size, pixel_size)
test_x  = torch.from_numpy(test_x)

# converting the target into torch format
test_y = test_y.astype(int);
test_y = torch.from_numpy(test_y)

# shape of training data
test_x.shape, test_y.shape

(torch.Size([5, 3, 400, 400]), torch.Size([5]))